In this Notebook I leave only baresoil samples and their reflectance values. Data has been downloaded from LUCAS 2015 dataset available on the internet. And sample points have been ingested in GEE to extract their spectral bands.

In [70]:
import pandas as pd
import numpy as np
from matplotlib.cbook import flatten

In [25]:
base_path = '/cats/datastore3/claraenv/code/my-projects/'  # '/home/clara/'  # '/cats/datastore3/claraenv/code/my-projects/'  # '/home/clara/Documents/'

## join original data and gee data

In [26]:
lucas = pd.read_csv(f'{base_path}/autoencoders/bare_soil-detection/data/topsoil_2015.csv')
lucas

,Point_ID,Coarse,Clay,Sand,Silt,pH_CaCl2,pH_H20,EC,OC,CaCO3,...,LU,NUTS_0,NUTS_1,NUTS_2,NUTS_3,LC0_Desc,LC1_Desc,LU1_Desc,x,y
0,26581768,NaN,NaN,NaN,NaN,6.9,7.24,18.14,24.6,34,...,U111,PT,PT1,PT15,PT150,Grassland,Grassland without tree/shrub cover,Agriculture (excluding fallow land and kitchen...,-8.775252,37.139104
1,26581792,NaN,NaN,NaN,NaN,5.4,5.81,14.81,21.9,0,...,U410,PT,PT1,PT15,PT150,Shrubland,Shrubland without tree cover,Abandoned areas,-8.833728,37.349747
2,26581954,NaN,NaN,NaN,NaN,7.0,7.29,28.10,18.4,46,...,U112,PT,PT1,PT17,PT170,Grassland,Spontaneously re-vegetated surfaces,Fallow land,-9.242092,38.768531
3,26601784,NaN,NaN,NaN,NaN,4.4,5.17,5.37,48.0,0,...,U120,PT,PT1,PT15,PT150,Shrubland,Shrubland with sparse tree cover,Forestry,-8.792255,37.283972
4,26601978,NaN,NaN,NaN,NaN,7.1,7.38,21.60,25.2,0,...,U120,PT,PT1,PT17,PT170,Woodland,Broadleaved woodland,Forestry,-9.282250,38.982827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21854,64881666,15.0,50.0,23.0,27.0,6.7,6.97,21.10,8.4,1,...,U112,CY,CY0,CY00,CY000,Bareland,Other bare soil,Fallow land,33.907085,34.989242
21855,64901668,3.0,32.0,39.0,28.0,7.3,7.79,10.84,10.8,2,...,U420,CY,CY0,CY00,CY000,Grassland,Spontaneously re-vegetated surfaces,Semi-natural and natural areas not in use,33.933925,35.000901
21856,64901672,4.0,34.0,43.0,23.0,7.3,7.80,15.09,6.7,8,...,U111,CY,CY0,CY00,CY000,Cropland,Barley,Agriculture (excluding fallow land and kitchen...,33.945599,35.035389
21857,64961676,17.0,10.0,79.0,11.0,7.4,8.05,15.50,5.7,3,...,U420,CY,CY0,CY00,CY000,Grassland,Spontaneously re-vegetated surfaces,Semi-natural and natural areas not in use,34.020365,35.053061


In [27]:
gee = pd.read_csv(f'{base_path}/autoencoders/bare_soil-detection/data/google-earth-engine/dataset_lucas.csv')
del gee['system:index']
gee

,B1,B1_1,B1_2,B1_3,B2,B2_1,B2_2,B2_3,B3,B3_1,...,B6,B6_1,B6_2,B6_3,B7,B7_1,B7_2,B7_3,Point_ID,.geo
0,1075.0,88.5,74.0,30.0,1046.0,138.0,121.0,47.0,1025.0,370.0,...,1310.0,986.0,783.0,405.0,867.0,424.5,302.0,155.0,48781732,"{""type"":""Point"",""coordinates"":[16.34733863,38...."
1,568.0,413.0,891.0,NaN,674.0,568.0,1207.0,NaN,1191.0,1025.0,...,2586.0,2146.0,4612.0,NaN,1947.0,1234.0,3225.0,NaN,31122190,"{""type"":""Point"",""coordinates"":[-4.58470017,41...."
2,858.0,774.5,955.5,723.0,1057.0,1045.0,1223.0,932.0,1672.0,1623.5,...,3707.0,3671.5,4413.5,3780.0,2590.0,2566.0,3096.0,2860.0,31741938,"{""type"":""Point"",""coordinates"":[-3.36152027,39...."
3,NaN,281.0,563.0,322.0,NaN,333.0,715.0,404.5,NaN,653.5,...,NaN,1655.0,3024.5,1968.5,NaN,1029.0,2414.0,1294.5,43122984,"{""type"":""Point"",""coordinates"":[9.87456666,49.9..."
4,NaN,168.5,716.0,329.0,NaN,238.0,899.0,408.0,NaN,500.0,...,NaN,1351.5,3411.0,1890.0,NaN,696.0,2659.5,1726.0,43302984,"{""type"":""Point"",""coordinates"":[10.12543334,49...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21854,-411.5,1266.0,65.0,345.0,-350.5,1244.0,104.5,426.0,-132.0,1209.0,...,297.0,715.0,576.0,591.0,170.0,541.0,231.5,443.0,46744374,"{""type"":""Point"",""coordinates"":[16.79063888,62...."
21855,367.5,301.0,409.5,283.0,458.5,381.0,539.5,363.0,750.5,755.0,...,2024.5,2094.0,2599.5,2171.5,1544.0,1325.0,1763.0,1412.0,49062684,"{""type"":""Point"",""coordinates"":[17.70032612,46...."
21856,260.5,254.0,537.0,204.0,324.0,460.0,606.0,294.0,536.5,930.0,...,1661.5,1882.0,1846.0,1840.0,923.0,1125.0,1105.0,984.0,32701972,"{""type"":""Point"",""coordinates"":[-2.31778558,40...."
21857,NaN,145.0,82.5,83.0,NaN,221.0,167.0,161.0,NaN,444.0,...,NaN,1205.0,1023.0,764.0,NaN,628.0,482.5,349.0,56342170,"{""type"":""Point"",""coordinates"":[25.76438587,41...."


In [28]:
data = pd.merge(lucas, gee, on="Point_ID")

## delete columns with undesired data and delete rows different from LC0_Desc == Bareland

In [29]:
data.columns

Index(['Point_ID', 'Coarse', 'Clay', 'Sand', 'Silt', 'pH_CaCl2', 'pH_H20',
       'EC', 'OC', 'CaCO3', 'P', 'N', 'K', 'LC', 'LU', 'NUTS_0', 'NUTS_1',
       'NUTS_2', 'NUTS_3', 'LC0_Desc', 'LC1_Desc', 'LU1_Desc', 'x', 'y', 'B1',
       'B1_1', 'B1_2', 'B1_3', 'B2', 'B2_1', 'B2_2', 'B2_3', 'B3', 'B3_1',
       'B3_2', 'B3_3', 'B4', 'B4_1', 'B4_2', 'B4_3', 'B5', 'B5_1', 'B5_2',
       'B5_3', 'B6', 'B6_1', 'B6_2', 'B6_3', 'B7', 'B7_1', 'B7_2', 'B7_3',
       '.geo'],
      dtype='object')

- LC0_Desc = Land cover description (ex. bareland)
- LC1_Desc = more land cover description (ex. other baresoil, sand, rock and stones)
- LU1_Desc = Land Use description (ex. agriculture, fallow land, sport...)
- LC = F10: rock and stones, F40: other baresoil, F20: sand

In [30]:
drop = ['Coarse', 'Clay', 'Sand', 'Silt', 'pH_CaCl2', 'pH_H20',
       'EC', 'OC', 'CaCO3', 'P', 'N', 'K', 'LU', 'NUTS_0', 'NUTS_1',
       'NUTS_2', 'NUTS_3']

In [31]:
data = data.drop(drop, axis=1).loc[(data['LC0_Desc']=='Bareland')]
data

,Point_ID,LC,LC0_Desc,LC1_Desc,LU1_Desc,x,y,B1,B1_1,B1_2,...,B5_3,B6,B6_1,B6_2,B6_3,B7,B7_1,B7_2,B7_3,.geo
18,26841758,F40,Bareland,Other bare soil,Fallow land,-8.466693,37.108232,297.0,905.5,904.0,...,4327.0,2103.0,4708.0,4714.0,2478.0,1162.5,3504.0,3489.5,1581.0,"{""type"":""Point"",""coordinates"":[-8.46669349,37...."
20,26841826,F40,Bareland,Other bare soil,Agriculture (excluding fallow land and kitchen...,-8.630850,37.705463,303.5,361.0,692.0,...,2713.0,2087.0,2377.5,4562.5,3158.0,1190.5,1459.0,3079.0,2117.0,"{""type"":""Point"",""coordinates"":[-8.63084954,37...."
36,27001936,F40,Bareland,Other bare soil,Fallow land,-8.726282,38.705168,460.0,964.0,1013.0,...,3039.5,2681.0,4132.0,4179.0,2644.5,1964.0,3268.0,3276.0,1881.5,"{""type"":""Point"",""coordinates"":[-8.72628189,38...."
49,27101940,F40,Bareland,Other bare soil,Fallow land,-8.624439,38.762267,352.0,335.0,545.0,...,3051.0,2415.0,2397.0,3034.0,1643.5,1523.0,1366.0,2129.0,804.0,"{""type"":""Point"",""coordinates"":[-8.62443932,38...."
89,27321784,F40,Bareland,Other bare soil,Agriculture (excluding fallow land and kitchen...,-8.001415,37.439943,245.0,606.0,601.0,...,1801.0,1832.0,3238.0,3971.0,2568.0,1022.0,2184.0,2712.0,1890.0,"{""type"":""Point"",""coordinates"":[-8.0014154,37.4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21766,57882508,F40,Bareland,Other bare soil,Agriculture (excluding fallow land and kitchen...,28.474431,44.040401,NaN,82.0,461.0,...,4694.5,NaN,1227.5,3103.5,3253.5,NaN,573.0,2043.0,2556.5,"{""type"":""Point"",""coordinates"":[28.47443109,44...."
21773,57922630,F40,Bareland,Other bare soil,Agriculture (excluding fallow land and kitchen...,28.884681,45.092388,NaN,352.0,554.0,...,1313.5,NaN,2486.0,2766.5,1976.0,NaN,1910.0,2573.5,1700.0,"{""type"":""Point"",""coordinates"":[28.8846806,45.0..."
21778,57962548,F40,Bareland,Other bare soil,Agriculture (excluding fallow land and kitchen...,28.688570,44.370723,NaN,339.5,608.5,...,2395.0,NaN,1681.5,2846.5,2584.5,NaN,1066.5,2390.0,2020.5,"{""type"":""Point"",""coordinates"":[28.6885701,44.3..."
21820,64161658,F40,Bareland,Other bare soil,Agriculture (excluding fallow land and kitchen...,33.126346,35.117963,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""type"":""Point"",""coordinates"":[33.12634593,35...."


In [32]:
# delete rest
drop = ['Point_ID', 'LC', 'LC0_Desc',  'LC1_Desc', 'LU1_Desc', 'x', 'y', '.geo']
data = data.drop(drop, axis=1)
data

,B1,B1_1,B1_2,B1_3,B2,B2_1,B2_2,B2_3,B3,B3_1,...,B5_2,B5_3,B6,B6_1,B6_2,B6_3,B7,B7_1,B7_2,B7_3
18,297.0,905.5,904.0,463.0,364.5,1051.0,1101.5,549.0,774.5,1646.0,...,3533.5,4327.0,2103.0,4708.0,4714.0,2478.0,1162.5,3504.0,3489.5,1581.0
20,303.5,361.0,692.0,459.0,349.0,574.5,930.0,571.0,778.5,991.0,...,3502.5,2713.0,2087.0,2377.5,4562.5,3158.0,1190.5,1459.0,3079.0,2117.0
36,460.0,964.0,1013.0,503.0,568.0,1113.0,1202.0,618.0,994.0,1595.0,...,3898.0,3039.5,2681.0,4132.0,4179.0,2644.5,1964.0,3268.0,3276.0,1881.5
49,352.0,335.0,545.0,199.5,432.0,423.0,651.0,256.5,687.0,745.0,...,3039.0,3051.0,2415.0,2397.0,3034.0,1643.5,1523.0,1366.0,2129.0,804.0
89,245.0,606.0,601.0,421.0,308.0,853.0,848.0,484.0,622.0,1142.0,...,3035.0,1801.0,1832.0,3238.0,3971.0,2568.0,1022.0,2184.0,2712.0,1890.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21766,NaN,82.0,461.0,3251.5,NaN,207.5,581.5,3306.5,NaN,932.0,...,3230.5,4694.5,NaN,1227.5,3103.5,3253.5,NaN,573.0,2043.0,2556.5
21773,NaN,352.0,554.0,417.5,NaN,463.5,628.5,444.5,NaN,851.0,...,1507.5,1313.5,NaN,2486.0,2766.5,1976.0,NaN,1910.0,2573.5,1700.0
21778,NaN,339.5,608.5,681.5,NaN,368.5,699.0,755.0,NaN,586.5,...,1982.0,2395.0,NaN,1681.5,2846.5,2584.5,NaN,1066.5,2390.0,2020.5
21820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# reshape: temporal samples x bands

In [74]:
# from axis0 = samples and axis1 = (bands x dates) TO  axis0 = (samples x dates) and axis1 = bands
data_array = np.array(data)
data_split = np.array(np.split(data_array, indices_or_sections = 7, axis = 1))
data_reshape = data_split.reshape(data_split.shape[0], data_split.shape[1]*data_split.shape[2])
data_transpose = np.transpose(data_reshape)
data_transpose.shape

(2416, 7)

In [75]:
# add temporal column
date = np.array(list(flatten([[1]*data_array.shape[0], [2]*data_array.shape[0], [3]*data_array.shape[0], [4]*data_array.shape[0]])))
date = date[..., np.newaxis]
all_data = np.concatenate([data_transpose, date], axis=1)
all_data.shape

(2416, 8)

In [77]:
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'date']
df = pd.DataFrame(all_data, columns = bands)
df

,B1,B2,B3,B4,B5,B6,B7,date
0,297.0,364.5,774.5,694.5,3143.5,2103.0,1162.5,1.0
1,905.5,1051.0,1646.0,2293.5,3429.5,4708.0,3504.0,1.0
2,904.0,1101.5,1822.0,2574.0,3533.5,4714.0,3489.5,1.0
3,463.0,549.0,1178.0,1026.0,4327.0,2478.0,1581.0,1.0
4,303.5,349.0,778.5,765.0,3184.5,2087.0,1190.5,1.0
...,...,...,...,...,...,...,...,...
2411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2413,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0


# save data

In [78]:
df.to_csv(f'{base_path}/autoencoders/bare_soil-detection/results/clean_data/bareland_LUCAS_2015.csv')

# End